In [1]:
from __future__ import division,unicode_literals
%matplotlib inline
import numpy as np
import pandas as pd
import json
import runProcs
from urllib.request import urlopen

import matplotlib.pyplot as plt

In [2]:
# 0. State abbreviations

# 0.1 dictionary:
stateAbbr = {
u'Alabama':u'AL',
u'Alaska':u'AK',
u'Arizona':u'AZ',
u'Arkansas':u'AR',
u'California':u'CA',
u'Colorado':u'CO',
u'Connecticut':u'CT',
u'Delaware':u'DE',
u'District of Columbia':u'DC',
u'Florida':u'FL',
u'Georgia':u'GA',
u'Hawaii':u'HI',
u'Idaho':u'ID',
u'Illinois':u'IL',
u'Indiana':u'IN',
u'Iowa':u'IA',
u'Kansas':u'KS',
u'Kentucky':u'KY',
u'Louisiana':u'LA',
u'Maine':u'ME',
u'Maryland':u'MD',
u'Massachusetts':u'MA',
u'Michigan':u'MI',
u'Minnesota':u'MN',
u'Mississippi':u'MS',
u'Missouri':u'MO',
u'Montana':u'MT',
u'Nebraska':u'NE',
u'Nevada':u'NV',
u'New Hampshire':u'NH',
u'New Jersey':u'NJ',
u'New Mexico':u'NM',
u'New York':u'NY',
u'North Carolina':u'NC',
u'North Dakota':u'ND',
u'Ohio':u'OH',
u'Oklahoma':u'OK',
u'Oregon':u'OR',
u'Pennsylvania':u'PA',
u'Rhode Island':u'RI',
u'South Carolina':u'SC',
u'South Dakota':u'SD',
u'Tennessee':u'TN',
u'Texas':u'TX',
u'Utah':u'UT',
u'Vermont':u'VT',
u'Virginia':u'VA',
u'Washington':u'WA',
u'West Virginia':u'WV',
u'Wisconsin':u'WI',
u'Wyoming':u'WY'
}

# 0.2 List of states in the US
stateList = [s for s in stateAbbr]

In [3]:
# 1. Construct series for price deflator

# 1.1 Obtain data from BEA
gdpDeflator = urlopen('http://bea.gov/api/data/?UserID=3EDEAA66-4B2B-4926-83C9-FD2089747A5B&method=GetData&datasetname=NIPA&TableID=13&Frequency=A&Year=X&ResultFormat=JSON&')

# result = gdpDeflator.readall().decode('utf-8')
result = gdpDeflator.read().decode('utf-8')
jsonResponse = json.loads(result)

In [4]:
# 1.2 Construct the data frame for the deflator series
values = []
years = []
for element in jsonResponse['BEAAPI']['Results']['Data']:
#     if element['LineDescription'] == 'Personal consumption expenditures':
    if element['LineDescription'] == 'Gross domestic product':
        years.append(element['TimePeriod'])
        values.append(float(element['DataValue'])/100)

values = np.array([values]).T
dataP = pd.DataFrame(values,index = years,columns = ['price level'])

# 1.3 Display the data
print(dataP)

      price level
1929      0.09896
1930      0.09535
1931      0.08555
1932      0.07553
1933      0.07345
1934      0.07749
1935      0.07908
1936      0.08001
1937      0.08347
1938      0.08109
1939      0.08033
1940      0.08131
1941      0.08680
1942      0.09369
1943      0.09795
1944      0.10027
1945      0.10288
1946      0.11618
1947      0.12887
1948      0.13605
1949      0.13581
1950      0.13745
1951      0.14716
1952      0.14972
1953      0.15157
1954      0.15298
1955      0.15559
1956      0.16091
1957      0.16625
1958      0.17001
...           ...
1987      0.59885
1988      0.61982
1989      0.64392
1990      0.66773
1991      0.68996
1992      0.70569
1993      0.72248
1994      0.73785
1995      0.75324
1996      0.76699
1997      0.78012
1998      0.78859
1999      0.80065
2000      0.81887
2001      0.83754
2002      0.85039
2003      0.86735
2004      0.89120
2005      0.91988
2006      0.94814
2007      0.97337
2008      0.99246
2009      1.00000
2010      

In [5]:
# 2. Construct series for per capita income by state, region, and the entire us

# 2.1 Obtain data from BEA
stateYpc = urlopen('http://bea.gov/api/data/?UserID=3EDEAA66-4B2B-4926-83C9-FD2089747A5B&method=GetData&datasetname=RegionalData&KeyCode=PCPI_SI&Year=ALL&GeoFips=STATE&ResultFormat=JSON&')
# result = stateYpc.readall().decode('utf-8')
result = stateYpc.read().decode('utf-8')
jsonResponse = json.loads(result)
# jsonResponse['BEAAPI']['Results']['Data'][0]['GeoName']

In [6]:
# 2.2 Construct the data frame for the per capita income series


# 2.2.1 Initialize the dataframe
regions = []
years = []
for element in jsonResponse['BEAAPI']['Results']['Data']:
    if element['GeoName'] not in regions:
        regions.append(element['GeoName'])
    if element['TimePeriod'] not in years:
        years.append(element['TimePeriod'])

df = np.zeros([len(years),len(regions)])
dataY = pd.DataFrame(df,index = years,columns = regions)
# 2.2.2 Populate the dataframe with values
for element in jsonResponse['BEAAPI']['Results']['Data']:
    try:
        dataY[element['GeoName']][element['TimePeriod']] = np.round(float(element[u'DataValue'])/float(dataP.loc[element['TimePeriod']]),2)# real
    except:
        dataY[element['GeoName']][element['TimePeriod']] = np.nan
        
# 2.2.3 Replace the state names in the index with abbreviations
columns=[]
for r in regions:
    if r in stateList:
        columns.append(stateAbbr[r])
    else:
        columns.append(r)
        
dataY.columns=columns

# 2.2.4 Display the data obtained from the BEA
dataY

,United States,AL,AK,AZ,AR,CA,CO,CT,DE,DC,...,WI,WY,New England,Mideast,Great Lakes,Plains,Southeast,Southwest,Rocky Mountain,Far West
1929,7063.46,3223.52,NaN,6042.85,3051.74,10034.36,6366.21,10398.14,10418.35,13369.04,...,6770.41,6790.62,8852.06,9822.15,8043.65,5709.38,3668.15,4749.39,5951.90,9145.11
1930,6512.85,2758.26,NaN,5432.62,2349.24,9323.54,6019.93,9711.59,8998.43,13780.81,...,6124.80,6103.83,8442.58,9334.03,7110.64,5285.79,3251.18,4174.10,5579.44,8515.99
1931,6171.83,2571.60,NaN,5026.30,2419.64,8801.87,5517.24,9421.39,9059.03,14646.41,...,5482.17,5575.69,8392.75,8930.45,6592.64,4862.65,3167.74,3892.46,4909.41,7983.64
1932,5322.39,2105.12,NaN,4249.97,2012.45,7732.03,4713.36,8301.34,7824.71,14656.43,...,4792.80,4964.91,7586.39,7851.18,5428.31,4130.81,2687.67,3296.70,4369.12,6964.12
1933,5105.51,2232.81,NaN,4206.94,2083.05,7474.47,4805.99,8032.68,7705.92,12865.90,...,4533.70,5064.67,7270.25,7420.01,5132.74,3771.27,2791.01,3349.22,4261.40,6739.28
1934,5510.39,2671.31,NaN,4671.57,2348.69,7807.46,4787.71,8530.13,8349.46,12427.41,...,4903.86,5342.62,7575.17,7768.74,5807.20,3987.61,3174.60,3639.18,4723.19,7175.12
1935,6031.87,2731.41,NaN,5260.50,2579.67,8409.21,5627.21,9041.48,8915.02,13037.43,...,5829.54,6297.42,7903.39,8168.94,6562.97,5096.11,3388.97,4046.54,5526.05,7713.71
1936,6736.66,3112.11,NaN,5774.28,3037.12,9711.29,6761.65,10186.23,10886.14,14285.71,...,6474.19,6899.14,8736.41,9148.86,7424.07,5174.35,3837.02,4524.43,6286.71,8923.88
1937,6936.62,3186.77,NaN,6050.08,3031.03,9608.24,6397.51,10410.93,11441.24,14496.23,...,6601.17,7260.09,8673.78,9188.93,7883.07,5702.65,3965.50,4899.96,6002.16,8841.50
1938,6548.28,2984.34,NaN,5907.02,2811.69,9618.94,6264.64,9606.61,9865.58,14070.79,...,6264.64,6905.91,8176.10,8755.70,7078.55,5401.41,3736.59,4809.47,6005.67,8842.03


In [7]:
# 3. State income data for 1840, 1880, and 1900

# 3.1.1 Import Easterlin's income data
easterlin_data = pd.read_csv('Historical Statistics of the US - Easterlin State Income Data.csv',index_col=0)

# 3.1.2 Import historic CPI data
historic_cpi_data=pd.read_csv('Historical Statistics of the US - cpi.csv',index_col=0)
historic_cpi_data = historic_cpi_data/historic_cpi_data.loc[1929]*float(dataP.loc['1929'])

In [8]:
# 3.2 Append to data beginning in 1929

# 3.2.1 Construct series for real incomes in 1840, 1880, and 1900
df_1840 = easterlin_data['Income per capita - 1840 - A [cur dollars]']/float(historic_cpi_data.loc[1840])
df_1880 = easterlin_data['Income per capita - 1880 [cur dollars]']/float(historic_cpi_data.loc[1890])
df_1900 = easterlin_data['Income per capita - 1900 [cur dollars]']/float(historic_cpi_data.loc[1900])

# 3.2.2 Put into a DataFrame and concatenate with previous
df = pd.DataFrame({'1840':df_1840,'1880':df_1880,'1900':df_1900}).transpose()
df = pd.concat([dataY,df]).sort_index()

In [9]:
# 4. Export data to csv
# series = dataY.sort_index()
series = df.sort_index()
dropCols = [u'AK', u'HI', u'New England', u'Mideast', u'Great Lakes', u'Plains', u'Southeast', u'Southwest', u'Rocky Mountain', u'Far West']
for c in dropCols:
    series = series.drop([c],axis=1)

series.to_csv('stateIncomeData.csv',na_rep='NaN')

In [10]:
# 4. Export notebook to .py
runProcs.exportNb('stateIncomeData')